<a href="https://colab.research.google.com/github/ashishsaini01/tenserflow_keras_prac/blob/master/Anomaly_detection_on_mnist_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
(x_train, _), (c_test, _) = tf.keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [13]:
# minimize the data and reshape it to 28*28*1
x_train = x_train.astype('float32') / 255.
x_test = c_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

In [27]:
# create the encoder and decoder

# pass the grey scale image
inputs = tf.keras.Input(shape = (28, 28, 1), name = 'input_layer')

# conv block1
encoded = tf.keras.layers.Conv2D(32, kernel_size  = 3, strides = 1, padding = 'same',  name = 'conv1')(inputs)
encoded = tf.keras.layers.BatchNormalization(name = 'batch_nor_1')(encoded)
encoded = tf.keras.layers.LeakyReLU(name = 'leaky_relu_1')(encoded)

# conv block 2
encoded = tf.keras.layers.Conv2D(64, kernel_size = 3, strides = 2, padding = 'same', name = 'conv2')(encoded)
encoded = tf.keras.layers.BatchNormalization(name = 'batch_norm2')(encoded)
encoded = tf.keras.layers.LeakyReLU(name = 'leaky_relu_2')(encoded)

# conv block 3
encoded = tf.keras.layers.Conv2D(64, kernel_size = 3, strides = 2, padding = 'same', name = 'conv3')(encoded)
encoded = tf.keras.layers.BatchNormalization(name = 'batch_norm3')(encoded)
encoded = tf.keras.layers.LeakyReLU(name = 'leaky_relu_3')(encoded)

# Decoder
# decode conv block 1
decoded = tf.keras.layers.Conv2DTranspose(64, kernel_size = 3, strides = 1, padding = 'same', name = 'deconv1')(encoded)
decoded = tf.keras.layers.BatchNormalization(name = 'batch_norm4')(decoded)
decoded = tf.keras.layers.LeakyReLU(name = 'leaky_relu_4')(decoded)

# decode conv block 2
decoded = tf.keras.layers.Conv2DTranspose(64, 3, strides = 2, padding = 'same', name = 'deconv2')(decoded)
decoded = tf.keras.layers.BatchNormalization(name = 'batch_norm5')(decoded)
deocded = tf.keras.layers.LeakyReLU(name = 'leaky_relu_5')(decoded)

# decode conv block 3
decoded = tf.keras.layers.Conv2DTranspose(32, 3, 2, padding = 'same', name = 'deconv3')(decoded)
decoded = tf.keras.layers.BatchNormalization(name = 'batch_norm6')(decoded)
decoded = tf.keras.layers.LeakyReLU(name = 'leaky_relu_6')(decoded)

# outputs
output = tf.keras.layers.Conv2DTranspose(1, 3, 1, padding = 'same', activation = 'sigmoid', name = 'output_conv')(decoded)

In [28]:
## ssim loss function
def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

In [30]:
autoencoder = tf.keras.Model(inputs, output)
optimizer = tf.keras.optimizers.Adam(lr = 0.0005)
autoencoder.compile(optimizer=optimizer, loss=SSIMLoss)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [31]:
hist=autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test)
                )

Epoch 1/10
469/469 [==============================] - 369s 780ms/step - loss: 0.1173 - val_loss: 0.5962
Epoch 2/10
469/469 [==============================] - 353s 754ms/step - loss: 8.0176e-04 - val_loss: 0.7440
Epoch 3/10
469/469 [==============================] - 354s 754ms/step - loss: 5.8993e-04 - val_loss: 0.7585
Epoch 4/10
469/469 [==============================] - 352s 751ms/step - loss: 4.4022e-04 - val_loss: 0.7383
Epoch 5/10
469/469 [==============================] - 353s 753ms/step - loss: 3.7902e-04 - val_loss: 0.7377
Epoch 6/10
469/469 [==============================] - 353s 753ms/step - loss: 3.4253e-04 - val_loss: 0.7541
Epoch 7/10
469/469 [==============================] - 354s 754ms/step - loss: 3.0680e-04 - val_loss: 0.8237
Epoch 8/10
469/469 [==============================] - 353s 752ms/step - loss: 2.4549e-04 - val_loss: 0.8571
Epoch 9/10
469/469 [==============================] - 349s 744ms/step - loss: 2.2289e-04 - val_loss: 0.8700
Epoch 10/10
469/469 [===========

In [34]:
# autoencoder.predict(x_test)

In [35]:
### references
# https://medium.com/analytics-vidhya/image-anomaly-detection-using-autoencoders-ae937c7fd2d1
# https://blog.keras.io/building-autoencoders-in-keras.html